# Projeto 1 - Ciência dos Dados

Nome: Guilherme dos Santos Martins

Nome: Giovana Cassoni Andrade

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [286]:
%matplotlib inline
# !pip install emoji
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import emoji
from emoji import UNICODE_EMOJI

Mostrando o diretório

In [287]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/giovanaandrade/Documents/Insper/Ciência dos Dados/Projeto 1/Projeto1_C.Dados-3


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [288]:
filename = 'Air Fryer.xlsx'

Dados do Treinamento

In [289]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,air fryer pra canhoto,1
1,@vanessapeitos @daizenbvc o franky conseguiria...,0
2,adoro poder aplicar meus conhecimentos de enge...,1
3,"@leilodecornohu1 ""amigo estamos falando de buc...",0
4,se eu n mandar isso aqui na dm de alguma esqui...,0


Tweets do treinamento

In [290]:
train.Treinamento

0                                  air fryer pra canhoto
1      @vanessapeitos @daizenbvc o franky conseguiria...
2      adoro poder aplicar meus conhecimentos de enge...
3      @leilodecornohu1 "amigo estamos falando de buc...
4      se eu n mandar isso aqui na dm de alguma esqui...
                             ...                        
295    @annaborgess air fryer eh foda simplesmente a ...
296    e @chromavka que escolheria brócolis na air fr...
297        estou morando em brasilia ou em uma air fryer
298    queria agradecer ao ser humano que inventou a ...
299    air fryer filha da puta, prega os trem tudo va...
Name: Treinamento, Length: 300, dtype: object

Probabilidades iniciais

In [291]:
probN = train.Relevância.value_counts(True)[0]
probR = train.Relevância.value_counts(True)[1]
probR, probN

(0.58, 0.42)

Dados do Teste

In [292]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@poetatulio @tatyene_mendes isso é uma xota ou...,0
1,"comprei uma air fryer p minha mãe, o bagulho caro",0
2,usando air fryer como se não houvesse o amanhã,1
3,minha mãe comprou uma air fryer e já ama mais ...,1
4,repita comigo: churrasco na air fryer não existe!,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto analisado foi a Air Fryer, fritadeira que não utiliza de óleo e que é provém de diversas marcas e podendo ter preços variados.
Nossa classificação sobre a relevância de um tweet foi se ele falava bem ou mal do nosso produto analisado, sendo bem o relevante e mal um irrelevante.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Criando as funções de cleanup:

In [293]:
import re 

def cleanup(text):
    punctuation = '[@!-.:?;()]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [294]:
def cleanup2(text):
    paragrafo = '\n'
    pattern = re.compile(paragrafo)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [295]:
def cleanup3(text):
    punctuation = '\/'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

Separando os tweets pela classificação da relevância

In [296]:
trainR = train.loc[train['Relevância'] == 1,:]
trainN = train.loc[train['Relevância'] == 0,:]

Transformando os tweets relevantes em string e adicionando a uma lista

In [297]:
trainR = trainR.Treinamento.apply(str)
trainR = trainR.to_list()

Transformando os tweets não relevantes em string e adicionando a uma lista

In [298]:
trainN = trainN.Treinamento.apply(str)
trainN = trainN.to_list()

Fazendo o cleanup em ambas as listas de tweets e o split

In [299]:
stringR = " "
for i in trainR:
    stringR = stringR + i + ' '
stringR = stringR.lower()
stringR = stringR.replace('air fryer', 'PRODUTO ')
stringR = cleanup3(cleanup2(cleanup(stringR)))
palavrasR = stringR.split()

In [300]:
stringN = " "
for i in trainN:
    stringN = stringN + i + ' '
stringN = stringN.lower()
stringN = stringN.replace('air fryer', 'PRODUTO ')
stringN = cleanup3(cleanup2(cleanup(stringN)))
palavrasN = stringN.split()

Criando duas Series, uma com todas as palavras dos tweets relevantes e outra com todas as palavras dos tweets não relevantes

In [301]:
serie_tweets_relevantes = pd.Series(palavrasR)
serie_tweets_nao_relevantes = pd.Series(palavrasN)

Contagem de cada uma das palavras de ambas as Series

In [302]:
tabela_tweets_relevantes = serie_tweets_relevantes.value_counts()
tabela_tweets_nao_relevantes = serie_tweets_nao_relevantes.value_counts()

Contagem dos valores relativos de ambas as Series

In [303]:
tabela_tweets_relevantes_relativa = serie_tweets_relevantes.value_counts(True)
tabela_tweets_nao_relevantes_relativa = serie_tweets_nao_relevantes.value_counts(True)

Transformando a tabela original Treinamento em string e lista para ter todos os tweets, não importando a classificação de relevância

In [304]:
train_str = train.Treinamento.apply(str)
train = train_str.to_list()

Fazendo o cleanup e split de todos os tweets

In [305]:
string = " "
for i in train:
    string = string + i + ' '
string = string.lower()
string = string.replace('air fryer', 'PRODUTO ')
string = cleanup3(cleanup2(cleanup(string)))
palavras = string.split()

Criando a Series com todas as palavras de todos os tweets

In [306]:
tabela_tweets = pd.Series(palavras)

Criando uma grande lista de pequenas listas, onde cada uma das pequenas listas contém um tweet. Nessa lista estão todos os tweets analisados.

In [307]:
lista_dos_tweets = []
lista_tweet = []
for tweet in train:
  lista_tweet.append(tweet)
  lista_dos_tweets.append(lista_tweet)
  lista_tweet = []


A etapa abaixo faz com que todas as palavras repetidas não sejam levadas em consideração, para não haver repetição.

In [308]:
palavras_nao_repetidas = []
for i in palavras:
    if i not in palavras_nao_repetidas:
        palavras_nao_repetidas.append(i)
tabela_nao_repetidas = pd.Series(palavras_nao_repetidas)

Percorrendo a lista de listas, fez a etapa do cleanup e do split em todos os tweets, criando uma grande lista de pequenas listas, nas quais possuem as palavras listadas.

In [309]:
i = 0
y = []
while i < 300:
   x = lista_dos_tweets[i]
   x = x[0].lower()
   x = x.replace('air fryer', 'PRODUTO ')
   x = cleanup3(cleanup2(cleanup(x)))
   x=x.split()
   y.append(x)
   i += 1

Em ambas as etapas abaixo, é realizado a suavização de Laplace, calculando a probabilidade de, levando em consideração os dados relevantes, ser uma frase relevante, e também calculando a probabilidade de, levando em consideração os dados não relevantes, ser uma frase não relevante:

In [310]:
i = 0
listaR = []
while i < 300:
    p = y[i]
    probFraseDadoR = 1
    for n in p:
        if n in tabela_tweets_relevantes:
            probFraseDadoR = probFraseDadoR * ((tabela_tweets_relevantes[n]+1)/(len(tabela_tweets_relevantes)+len(tabela_nao_repetidas)))
        else:
            probFraseDadoR = probFraseDadoR * (1/(len(tabela_tweets_relevantes)+len(tabela_nao_repetidas)))
    listaR.append(probFraseDadoR)
    i += 1

In [311]:
i = 0
listaN = []
while i < 300:
    p = y[i]
    probFraseDadoN = 1
    for n in p:
        if n in tabela_tweets_nao_relevantes:
            probFraseDadoN = probFraseDadoN * ((tabela_tweets_nao_relevantes[n]+1)/(len(tabela_tweets_nao_relevantes)+len(tabela_nao_repetidas)))
        else:
            probFraseDadoN = probFraseDadoN * (1/(len(tabela_tweets_nao_relevantes)+len(tabela_nao_repetidas)))
    listaN.append(probFraseDadoN)
    i += 1

Em ambas as etapas abaixo, calcula-se a probabilidade de, dado uma frase, ela ser relevante, e também calcula-se a probabilidade de, dado uma frase não relevante, ela ser não relevante:

In [312]:
i = 0
listar = []
while i < 300:
    probRDadoFrase = listaR[i] * probR
    listar.append(probRDadoFrase)
    i+=1

In [313]:
i = 0
listan = []
while i < 300:
    probNDadoFrase = listaN[i] * probN
    listan.append(probNDadoFrase)
    i+=1

Criando uma lista com os níveis de relevância dos tweets do excel Treinamento.

In [314]:
i = 0
relevancia = []
while i < 300:
    if listar[i] > listan[i]:
        relevancia.append(1)
    else:
        relevancia.append(0)
    i += 1


Criando uma função com todos esses passos realizados acima.

In [315]:
def relevante(lista_de_tweet):
    lista_dos_tweets2 = []
    lista_tweet2 = []
    for tweet in lista_de_tweet:
        lista_tweet2.append(tweet)
        lista_dos_tweets2.append(lista_tweet2)
        lista_tweet2 = []

    i = 0
    y1 = []
    while i < 200:
        x = lista_dos_tweets2[i]
        x = x[0].lower()
        x = x.replace('air fryer', 'PRODUTO ')
        x = cleanup3(cleanup2(cleanup(x)))
        x=x.split()
        y1.append(x)
        i += 1
        
    i = 0
    listaR = []
    while i < 200:        
        p = y1[i]
        probFraseDadoR = 1
        for n in p:
            if n in tabela_tweets_relevantes:
                probFraseDadoR = probFraseDadoR * ((tabela_tweets_relevantes[n]+1)/(len(tabela_tweets_relevantes)+len(tabela_nao_repetidas)))
            else:
                probFraseDadoR = probFraseDadoR * (1/(len(tabela_tweets_relevantes)+len(tabela_nao_repetidas)))
        listaR.append(probFraseDadoR)
        i += 1

    i = 0
    listaN = []
    while i < 200:       
        p = y1[i]
        probFraseDadoN = 1
        for n in p:
            if n in tabela_tweets_nao_relevantes:
                probFraseDadoN = probFraseDadoN * ((tabela_tweets_nao_relevantes[n]+1)/(len(tabela_tweets_nao_relevantes)+len(tabela_nao_repetidas)))
            else:
                probFraseDadoN = probFraseDadoN * (1/(len(tabela_tweets_nao_relevantes)+len(tabela_nao_repetidas)))
        listaN.append(probFraseDadoN)
        i += 1

    i = 0
    listar = []
    while i < 200:        
        probRDadoFrase = listaR[i] * probR
        listar.append(probRDadoFrase)
        i += 1

    i = 0
    listan = []
    while i < 200:
        probNDadoFrase = listaN[i] * probN
        listan.append(probNDadoFrase)
        i += 1

    i = 0
    relevancia = []
    while i < 200:        
        if listar[i] > listan[i]:
            relevancia.append(1)
        else:
            relevancia.append(0)
        i += 1
    return relevancia

Novamente transformando o excel Treinamento em uma lista de strings

In [316]:
train1 = pd.read_excel(filename)
train2 = train1.Treinamento.apply(str)
train3 = train2.to_list()

Utilizando a função (de relevância) criada na lista de strings (x1), retornando uma lista com as classificações obtidas. Fazendo uma lista com as classificações atribuídas pelos integrantes do grupo (vreal1).

In [317]:
x1 = relevante(train3)
v1 = pd.Series(x1)
vreal1 = train1.Relevância.to_list()

Fazendo uma comparação para classificar e, posteriormente, extrair as contagens

In [318]:
lista_porcentagens1 = [0] * len(x1)
i = 0
while i < len(x1):
    if x1[i] == vreal1[i] and x1[i] == 0:
        lista_porcentagens1[i] = 'verdadeiro negativo'
    if x1[i] == vreal1[i] and x1[i] == 1:
        lista_porcentagens1[i] = 'verdadeiro positivo'
    if x1[i] != vreal1[i] and x1[i] == 0:
        lista_porcentagens1[i] = 'falso negativo'
    if x1[i] != vreal1[i] and x1[i] == 1:
        lista_porcentagens1[i] = 'falso positivo'
    i += 1

porcentagens1 = pd.Series(lista_porcentagens1)
porcentagens1

0      verdadeiro positivo
1      verdadeiro negativo
2      verdadeiro positivo
3      verdadeiro negativo
4      verdadeiro negativo
              ...         
195    verdadeiro positivo
196    verdadeiro positivo
197    verdadeiro positivo
198    verdadeiro negativo
199    verdadeiro negativo
Length: 200, dtype: object

Extraindo as contagens obtidas:

In [319]:
porcentagens1.value_counts(True) * 100

verdadeiro positivo    59.0
verdadeiro negativo    37.0
falso positivo          4.0
dtype: float64

Mensagens corretamente classificadas: Acurácia

In [320]:
acuracia1 = 59 + 37
acuracia1 = str(acuracia1) + ' %'
acuracia1

'96 %'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

Transformando o excel Teste em uma lista de strings

In [321]:
test1 = test.Teste.apply(str)
test2 = test1.to_list()

Utilizando a função (de relevância) criada na lista de strings (x), retornando uma lista com as classificações obtidas. Fazendo uma lista com as classificações atribuídas pelos integrantes do grupo (vreal).

In [322]:
x = relevante(test2)
v = pd.Series(x)
vreal = test.Relevância.to_list()

Fazendo uma comparação para classificar e, posteriormente, extrair as contagens

In [323]:
lista_porcentagens = [0] * len(x)
i = 0
while i < len(x):
    if x[i] == vreal[i] and x[i] == 0:
        lista_porcentagens[i] = 'verdadeiro negativo'
    if x[i] == vreal[i] and x[i] == 1:
        lista_porcentagens[i] = 'verdadeiro positivo'
    if x[i] != vreal[i] and x[i] == 0:
        lista_porcentagens[i] = 'falso negativo'
    if x[i] != vreal[i] and x[i] == 1:
        lista_porcentagens[i] = 'falso positivo'
    i += 1

porcentagens = pd.Series(lista_porcentagens)
porcentagens

0      verdadeiro negativo
1           falso positivo
2      verdadeiro positivo
3      verdadeiro positivo
4           falso positivo
              ...         
195    verdadeiro positivo
196    verdadeiro positivo
197         falso positivo
198         falso positivo
199         falso positivo
Length: 200, dtype: object

Extraindo as seguintes contagens:

- Porcentagem de verdadeiros positivos
- Porcentagem de falsos positivos
- Porcentagem de verdadeiros negativos
- Porcentagem de falsos negativos

In [324]:
porcentagens.value_counts(True) * 100

verdadeiro positivo    48.5
falso positivo         31.0
verdadeiro negativo    15.0
falso negativo          5.5
dtype: float64

Mensagens corretamente classificadas: Acurácia

In [325]:
acuracia = 48.5 + 15.0
acuracia = str(acuracia) + ' %'
acuracia

'63.5 %'

___
## Concluindo

Após o classificador ser utilizado, pode-se ver que ele obteve uma performance mediana, com uma acurácia de 63.5 % quando utilizado na planilha do excel de teste.

Com esses dados, pode-se entender que essa plataforma de classificação tem um índice de acertos satisfatório. Tendo em vista que a fonte de dados analisada pelo classificador pode ter, e nesse caso tem, novas palavras que não foram inseridas no treinamento dele, uma acurácia de mais da metade de seus itens reflete que a estratégia utilizada pelo grupo atendeu às expectativas propostas nesse projeto.

Porém, o grupo também entende que novas melhorias seriam necessárias para uma utilização realmente prática do nosso produto. Como por exemplo, um meio de adicionar à base de dados novas palavras e seus sentidos para relacioná-los à base de dados já existente.

Em relação à como nosso classificador Naive-Baiss analisa mensagens com duplo sentido, dupla negação ou sarcasmo, ele não tem uma ferramenta voltada para esse sentido, sendo esse um dos motivos para uma acurácia menor do que a esperada (um nível mais que satisfatório seria acima de 75% de acurácia).

## Comparando o Treinamento e o Teste:

Na análise do Treinamento, foram obtidos os seguintes resultados:

- verdadeiro positivo: 59.0 %
- verdadeiro negativo: 37.0 %
- falso positivo: 4.0 %

Na análise do Teste, foram obtidos os seguintes resultados:

- verdadeiro positivo: 48.5 %
- falso positivo: 31.0 %
- verdadeiro negativo: 15.0 %
- falso negativo: 5.5 %

Com esses dados pode-se inferir que:

- em ambas as análises, foram poucas (ou nula) as mensagens classificadas como falsos negativos;
- a porcentagem de mensagens falsos positivos aumentou surpreendemente;
- as porcentagens das mensagens verdadeiros positivos e das mensagens verdadeiros negativos abaixou;

Muitas dessas alterações no resultado podem ter sido resultado de mensagens com duplo sentido, dupla negação ou sarcasmo, uma vez que esse classificador não capta essas nuances da língua humana.

## Propondo um plano de expansão

Para a expansão do nosso projeto como um todo, várias propostas podem ser colocadas em prática, porém as mais viáveis são de analisar o sentido de cada palavra em uma espécie de plataforma de pesquisa e assim poder analisar novas palavras de forma que a acurácia não diminua.
Isso tendo em vista que ainda não é utilizada tal proposta pois ao não se ter anteriormente uma análise humana de quais palavras estão em tweets relevantes, termos seriam colocados aleatoriamente, atrapalhando ainda mais a performance do classificador.

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**